### Compare Imputation scores

This is an investigation to determine which method is best for filling in the fairly large number of missing Age values.
I will compare:

- Filling in the mean value of all ages
- Filling in the median value grouped by Sex, Title and pclass
- Imputation using MICE (Multivariate/Multiple Imputation by Chained Equations)

First get the data (which has already some cleaning applied) and import the correct libraries:

In [32]:
# Import libraries

import numpy as np
from numpy.random import random_integers
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

print('Libraries Ready!')

Libraries Ready!


In [33]:
# Load the full training and test dataset, saved to csv on day58
full = pd.read_csv('/home/sophie/Python/notechain/Titanic/data/061_full.csv', header=0) 

# current list of variables
print(full.dtypes)

print(full.head(4))

Age             float64
Parch           float64
Pclass          float64
SibSp           float64
FamSize         float64
Sex_female      float64
Sex_male        float64
Embarked_C      float64
Embarked_Q      float64
Embarked_S      float64
Title_Master    float64
Title_Miss      float64
Title_Mr        float64
Title_Mrs       float64
Title_Rare      float64
dtype: object
    Age  Parch  Pclass  SibSp  FamSize  Sex_female  Sex_male  Embarked_C  \
0  22.0    0.0     3.0    1.0      2.0         0.0       1.0         0.0   
1  38.0    0.0     1.0    1.0      2.0         1.0       0.0         1.0   
2  26.0    0.0     3.0    0.0      1.0         1.0       0.0         0.0   
3  35.0    0.0     1.0    1.0      2.0         1.0       0.0         0.0   

   Embarked_Q  Embarked_S  Title_Master  Title_Miss  Title_Mr  Title_Mrs  \
0         0.0         1.0           0.0         0.0       1.0        0.0   
1         0.0         0.0           0.0         0.0       0.0        1.0   
2         0.0    

### Impute Age from Mean Age

In [34]:
# rename the dataframe
full_mean = full

full_mean['Age'].fillna(full_mean['Age'].mean(), inplace = True)

print(full_mean['Age'][0:10])

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.881138
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: Age, dtype: float64


### Impute Age based on Sex, Title and Pclass

Best to import the data again with the columns in categories


In [35]:
# Load the full training and test dataset, saved to csv on day58
df = pd.read_csv('/home/sophie/Python/notechain/Titanic/data/058_full.csv', header=0)

print(list(full))

['Age', 'Parch', 'Pclass', 'SibSp', 'FamSize', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare']


This is code copied straight from [052-Titanic](052-Titanic_new.ipynb):

In [36]:
# Make an array where null values are False.
mask_Age = df.Age.notnull()

# New dataframe where all rows have a value for age. 
Age_Sex_Title_Pclass = df.loc[mask_Age, ["Age", "Title", "Sex", "Pclass"]]

# Groupby object to group by Title, Pclass and Sex
Filler_Ages_1 = Age_Sex_Title_Pclass.groupby(by = ["Title", "Pclass", "Sex"]).median()

# This moves both Sex and Pclass into column headers and does so in that order. 
Filler_Ages = Filler_Ages_1.Age.unstack(level = -1).unstack(level = -1)

mask_Age = df.Age.isnull()  # A mask where null values are True

# New DataFrame with missing values for age
Age_Sex_Title_Pclass_missing = df.loc[mask_Age, ["Title", "Sex", "Pclass"]]

In [38]:
# Look-up function for the calculated median ages. 
def Age_filler(row):
    if row.Sex == "female":
        age = Filler_Ages.female.loc[row["Title"], row["Pclass"]]
        return age
    elif row.Sex == "male":
        age = Filler_Ages.male.loc[row["Title"], row["Pclass"]]
        return age
    
# Make a new column on "missing" dataframe and add the median value to each row. 
Age_Sex_Title_Pclass_missing["Age"] = Age_Sex_Title_Pclass_missing.apply(Age_filler, axis= 1)

# reform the 'Age' column.
df["AgeFill"] = pd.concat([Age_Sex_Title_Pclass["Age"], Age_Sex_Title_Pclass_missing["Age"]])

In [39]:
# use the factorized "full" dataset and add our new age column to it

full3cols = pd.concat([full,df['AgeFill']], axis=1) 

print(list(full3cols))

['Age', 'Parch', 'Pclass', 'SibSp', 'FamSize', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'AgeFill']


In [40]:
# Drop the old 'Age' column

full3cols = full3cols.drop(['Age'], axis=1)

print(list(full3cols))

['Parch', 'Pclass', 'SibSp', 'FamSize', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'AgeFill']


### Impute Age using MICE

We need to get the dataframe into numerical values. This means using dummies to transform categorical variables into 1s and 0s.